In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix2/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix2/dw_matrix_car


In [27]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

### Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    # przypisanie id do każdego koloru
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')))

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  # neg_mean_absolute_error = wyniki, które dostajemy, będą też miały wartość ujemną
  scores = cross_val_score(model, X, y, cv = 3, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [32]:
feats = ["param_napęd__cat","param_skrzynia-biegów__cat","param_faktura-vat__cat","param_rok-produkcji","param_stan__cat","feature_kamera-cofania__cat","param_moc","param_pojemność-skokowa","feature_czujniki-parkowania-przednie__cat","feature_regulowane-zawieszenie__cat","feature_wspomaganie-kierownicy__cat","feature_asystent-pasa-ruchu__cat","feature_światła-led__cat","feature_łopatki-zmiany-biegów__cat","seller_name__cat","feature_bluetooth__cat","param_marka-pojazdu__cat","feature_system-start-stop__cat","feature_światła-do-jazdy-dziennej__cat","param_kraj-pochodzenia__cat"]

xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), feats)

[21:16:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:16:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:16:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9951.226122123444, 69.88263058589365)

### Hyperopt

In [33]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  # Zwracamy słownik

  # status_OK - mówi o tym, że udało się wyliczyć iterację (udało się skończyć zgodnie z planem)
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}


# space - informacja dla optymalizatora, gdzie najlepiej się poruszać
xgb_reg_params = {
    # parametry dynamiczne
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype = int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    # parametry statyczne
    'objective':        'reg:squarederror',
    'seed': 0,
}

## run
best = fmin(obj_func, xgb_reg_params, algo = tpe.suggest, max_evals = 10)

best

Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.25, 'max_depth': 15, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 9, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.25, 'max_depth': 15, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.2, 'max_depth': 5, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 14, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'max_depth': 10, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Trainin

{'colsample_bytree': 0.9,
 'learning_rate': 1,
 'max_depth': 6,
 'subsample': 0.75}